In [0]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
from IPython import display; display.clear_output()
from fastai.tabular import *
import pandas as pd, numpy as np, matplotlib.pyplot as plt

df = pd.read_csv('doctor_train.csv')
test_df = pd.read_csv('doctor_test.csv')

Xdf = df.drop('Y',axis=1)
Ydf = df['Y']

Xdf = pd.get_dummies(Xdf)
test_df = pd.get_dummies(test_df)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(Ydf)
Xdf['Y'] = le.transform(Ydf)

In [0]:
procs = [FillMissing, Categorify, Normalize]
num_cal = ['age', 'Money', 'day', 'Time', 'Doctor_visits', 'last_visit',
       'cured_in', 'Profession_admin.', 'Profession_blue-collar',
       'Profession_entrepreneur', 'Profession_housemaid',
       'Profession_management', 'Profession_retired',
       'Profession_self-employed', 'Profession_services', 'Profession_student',
       'Profession_technician', 'Profession_unemployed', 'Profession_unknown',
       'Status_divorced', 'Status_married', 'Status_single', 'edu_primary',
       'edu_secondary', 'edu_tertiary', 'edu_unknown', 'Irregular_no',
       'Irregular_yes', 'residence_no', 'residence_yes', 'prev_diagnosed_no',
       'prev_diagnosed_yes', 'communication_cellular',
       'communication_telephone', 'communication_unknown', 'Month_apr',
       'Month_aug', 'Month_dec', 'Month_feb', 'Month_jan', 'Month_jul',
       'Month_jun', 'Month_mar', 'Month_may', 'Month_nov', 'Month_oct',
       'Month_sep', 'side_effects_failure', 'side_effects_other',
       'side_effects_success', 'side_effects_unknown']
cat_col = []
dep_var = ['Y']

data = (TabularList.from_df(Xdf, procs=procs, cont_names=num_cal, cat_names=cat_col)
          .split_by_idx(valid_idx=range(int(len(Xdf)*0.9),len(Xdf)))
          .label_from_df(cols=dep_var)
          .add_test(TabularList.from_df(test_df, cont_names=num_cal, cat_names=cat_col, procs=procs))
          .databunch())

In [20]:
learn = tabular_learner(data, layers=[1000,500,100, 50], metrics=accuracy)
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.240483,0.216814,0.902837,00:12
1,0.225900,0.208799,0.905319,00:12
2,0.216765,0.201385,0.910638,00:12


In [21]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)
pred_prob, pred_class = preds.max(1)

le_y = le.inverse_transform(pred_class)

output = pd.DataFrame({'id': test_df.ID,
                       'Y': le_y})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')